# Bleaching samples

## update sample type 
- everything is healthy or diseased right now, 
-> change to bleached based sampling notes
-> then crosscheck with colonydata?

In [1]:
import numpy as np
import pandas as pd
import os
import re

In [2]:
os.getcwd() 

'/Users/brookesienkiewicz/Documents/Code_notebook/QA_QC/SampleData'

In [3]:
# upload sample and colony data 
sample=pd.read_csv('~/Documents/sctld/SCTLD_samples/Sample_Data/CBC_samples.csv')
colony=pd.read_csv('~/Documents/sctld/SCTLD_samples/Sample_Data/CBC_ColonyData.csv')
colony.drop(columns="Unnamed: 0",inplace=True)

In [4]:
sample.shape

(2529, 20)

### Sample Fixes 
- first fix bad formatting/syntax 

In [5]:
# make all species names uniform 
sample['Species'].unique()

['OFAV' 'SSID' 'PAST' 'PSTR' 'MCAV' 'OANN' 'DLAB' 'CNAT' 'DL' 'MMEA'
 'PAST ' 'Unknown' 'OFAV ']


In [6]:
# Change OFAV/OANN variations to ORBI
# already did this one 

# sample.loc[:,'Species']=sample['Species'].str.replace('OANN/OFAV?',"ORBI")
# sample.loc[:,'Species']=sample['Species'].str.replace('OFAV/OANN',"ORBI")

# Fix space in PAST in OFAV
sample.loc[:,'Species']=sample['Species'].str.replace('PAST ',"PAST")
sample.loc[:,'Species']=sample['Species'].str.replace('OFAV ',"OFAV")

# Change 'DL' -> DLAB 
sample['Species'] = sample['Species'].replace('\\bDL\\b', 'DLAB', regex=True)

In [7]:
# Check that changes worked and store specie names in list
species_list=sample['Species'].unique()
print(species_list)

['OFAV' 'SSID' 'PAST' 'PSTR' 'MCAV' 'OANN' 'DLAB' 'CNAT' 'MMEA' 'Unknown']


### Add Bleaching sample type 

In [11]:
# changed my mind..just replacing with colony health statuses since those are being reviewed 

# search for sampling notes with clp or clb 

#if col contains clb or clp 
# replace status with bleached tissue 
# but don't replace if sample type is diseased, want to manually look at those 

# sample.loc[
#     sample['Sampling_notes'].str.contains('CLP|CLB', case=False, na=False) &
#     ~sample['Health_status'].str.contains('Diseased',case=False),
#     'Health_status']= 'Bleached_Tissue'

# (code is in place replacement)

In [12]:
# match sample type to colony statuses
# got from my cbc_metagenomics code

In [13]:
# add colony ID - t# newtagnum species
sample = sample.copy()
sample['TransectNum_str'] = 'T' + sample['TransectNum'].astype(str)
sample['colony_id'] = sample[['TransectNum_str', 'NewTagNum', 'Species']].astype(str).agg('_'.join, axis=1)
sample.drop(columns='TransectNum_str', inplace=True)

In [14]:
# add colony ID - t# newtagnum species
colony = colony.copy()
colony['TransectNum_str'] = 'T' + colony['TransectNum'].astype(str)
colony['colony_id'] = colony[['TransectNum_str', 'NewTagNum', 'Species']].astype(str).agg('_'.join, axis=1)
colony.drop(columns='TransectNum_str', inplace=True)

In [15]:
## BELOW SCRIPT IS IN PROGRESS 8/20/2025 

In [16]:
sample['Sample_type'].unique()

array(['Core_frozen', 'Core_RNAlater', 'Core_EtOH', 'Probiotics',
       'Immune', 'TEM', nan, 'Syringe'], dtype=object)

In [17]:
# filter samples
sample_subset=sample[
    (sample['Sample_type'] == 'Core_EtOH') |
    (sample['Sample_type'] == 'Core_RNAlater') |
    (sample['Sample_type'] == 'Core_frozen') |
    (sample['Sample_type'] == 'Immune') 
]

In [18]:
# extract condition columns 
cond_cols=['colony_id'] + colony.columns[colony.columns.str.contains('_Condition')].tolist()

In [19]:
# make merged df for everything

# filter colony data for conditions 
small_colony=colony.loc[:,cond_cols]

# and pivot
conditions_long=small_colony.melt(id_vars='colony_id', 
                          var_name='Month_year', 
                          value_name='colony_condition')

# extract month_year from column names to match sample data format
conditions_long['Month_year'] = conditions_long['Month_year'].str.extract(r'(\d+)').astype(int)
# add back in colony data 
cols_keep=['colony_id','Date_InitialTag','Transect','Species','immune_y/n','Date_DocumentedMortality']
colony_filtered=conditions_long.merge(colony[cols_keep],
                                      on='colony_id', 
                                      how='left')

# filter sample data (USING SUBSET FOR NOW) 
filtered_samples=sample_subset.loc[:,('colony_id','Month_year','Health_status','Sampling_notes','Tubelabel_species','Sample_type')] 
# rename column for sample health statuses 
filtered_samples = filtered_samples.rename(columns={'Health_status': 'sample_condition'})

# merge colony and sample dfs by colony id and monthyear
merged = pd.merge(
    colony_filtered,
    filtered_samples,
    on=['colony_id', 'Month_year'],
    how='outer'
)

# remove rows with no sample 
merged=merged[merged['Tubelabel_species'].notna()]

In [21]:
merged.loc[(merged['colony_id']=='T1_2_PAST') & 
    (merged['Month_year']==92023),:]

,colony_id,Month_year,colony_condition,Date_InitialTag,Transect,Species,immune_y/n,Date_DocumentedMortality,sample_condition,Sampling_notes,Tubelabel_species,Sample_type
419,T1_2_PAST,92023,"CLP,CLB",6/21/19,CBC30N,PAST,y,Healthy,Healthy,CLP 90%,092023_BEL_CBC_T1_171_PAST,Core_RNAlater
420,T1_2_PAST,92023,"CLP,CLB",6/21/19,CBC30N,PAST,y,Healthy,Healthy,CLP 90%,092023_BEL_CBC_T1_221_PAST,Core_frozen
421,T1_2_PAST,92023,"CLP,CLB",6/21/19,CBC30N,PAST,y,Healthy,Healthy,CLP 90%,092023_BEL_CBC_T1_251_PAST,Immune


In [22]:
# make merged df for everything (not just by species) and use code below 

# if sample id is not na, colony condition is CLP/CLB and sample status is healthy, 
# replace sample status to bleached_tissue 
# then somehow replace original sample data? maybe just by matching tubelabel?

bl_conds = ['CLP','CLB','CLP,CLB','CLB,CLP']
disease= ['Diseased_Tissue', 'Diseased_Margin']

# get rows with no samples 
# rows_na = (merged['sample_ids'].explode().isna().groupby(level=0).any())

# get rows with clp/clb conditions 
rows = (merged['colony_condition'].isin(bl_conds) &
        # exclude rows with no samples 
        # merged['Tubelabel_species'].notna() # (already added this during merging)
        
        # only when condition is healthy (don't want to override diseased-tissue)
        ~merged['sample_condition'].isin(disease))

# update only those rows with bleached sample status 
merged.loc[rows, 'sample_condition'] = 'Bleached_Tissue'

In [24]:
merged.loc[(merged['colony_id']=='T1_2_PAST') & 
    (merged['Month_year']==92023),:]
# check it worked

,colony_id,Month_year,colony_condition,Date_InitialTag,Transect,Species,immune_y/n,Date_DocumentedMortality,sample_condition,Sampling_notes,Tubelabel_species,Sample_type
419,T1_2_PAST,92023,"CLP,CLB",6/21/19,CBC30N,PAST,y,Healthy,Bleached_Tissue,CLP 90%,092023_BEL_CBC_T1_171_PAST,Core_RNAlater
420,T1_2_PAST,92023,"CLP,CLB",6/21/19,CBC30N,PAST,y,Healthy,Bleached_Tissue,CLP 90%,092023_BEL_CBC_T1_221_PAST,Core_frozen
421,T1_2_PAST,92023,"CLP,CLB",6/21/19,CBC30N,PAST,y,Healthy,Bleached_Tissue,CLP 90%,092023_BEL_CBC_T1_251_PAST,Immune


In [26]:
# check for duplicates in merged
dup_mask = merged.duplicated(subset=['colony_id','Tubelabel_species','Month_year','Sample_type'], keep=False)
duplicates = merged[dup_mask]

duplicates

,colony_id,Month_year,colony_condition,Date_InitialTag,Transect,Species,immune_y/n,Date_DocumentedMortality,sample_condition,Sampling_notes,Tubelabel_species,Sample_type


In [31]:
# export changes to sctld_samples
test=sample.merge(
    merged[['colony_id','Tubelabel_species','sample_condition','Month_year','Sample_type']],
    on=['colony_id', 'Tubelabel_species','Month_year','Sample_type'],
    how='left')
test.head()

,Month_year,Country,Location,CollectionDate,Transect,TransectNum,OldTagNum,NewTagNum,Species,Time_sampled,...,SampleNum,Health_status,Sampling_notes,Tubelabel_species,Sample_physical_location,Extraction_physical_location,Date_sequenced,Notes,colony_id,sample_condition
0,122022,BEL,CBC,12/5/22,CURLEW,4,NaN,77,OFAV,NaN,...,18,Healthy,NaN,122022_BEL_CBC_T4_18_OFAV,NaN,NaN,NaN,NaN,T4_77_OFAV,Healthy
1,92023,BEL,CBC,9/27/23,CBC30N,1,NaN,1,SSID,NaN,...,185,Diseased_Margin,only margin sample available,092023_BEL_CBC_T1_185_SSID,UML_NARWHAL_R1_B10,NaN,NaN,NaN,T1_1_SSID,Diseased_Margin
2,92023,BEL,CBC,9/25/23,CBC30N,1,NaN,2,PAST,NaN,...,171,Healthy,CLP 90%,092023_BEL_CBC_T1_171_PAST,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,NaN,NaN,T1_2_PAST,Bleached_Tissue
3,92023,BEL,CBC,9/25/23,CBC30N,1,NaN,3,SSID,NaN,...,173,Healthy,CLP 80%; DC 20%,092023_BEL_CBC_T1_173_SSID,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,NaN,NaN,T1_3_SSID,Healthy
4,92023,BEL,CBC,9/25/23,CBC30N,1,NaN,12,PSTR,NaN,...,177,Healthy,No CL,092023_BEL_CBC_T1_177_PSTR,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,NaN,NaN,T1_12_PSTR,Healthy


In [32]:
test.shape

(2529, 22)

In [34]:
# update only where there's a change
rows = test['sample_condition'].notna() & (test['Health_status'] != test['sample_condition'])

test.loc[rows, 'Health_status'] = test.loc[rows, 'sample_condition']

In [35]:
# drop new cols
test = test.drop(columns=['sample_condition','colony_id'])

In [36]:
test.head()

,Month_year,Country,Location,CollectionDate,Transect,TransectNum,OldTagNum,NewTagNum,Species,Time_sampled,Time_processed,Sample_type,SampleNum,Health_status,Sampling_notes,Tubelabel_species,Sample_physical_location,Extraction_physical_location,Date_sequenced,Notes
0,122022,BEL,CBC,12/5/22,CURLEW,4,NaN,77,OFAV,NaN,NaN,Core_frozen,18,Healthy,NaN,122022_BEL_CBC_T4_18_OFAV,NaN,NaN,NaN,NaN
1,92023,BEL,CBC,9/27/23,CBC30N,1,NaN,1,SSID,NaN,NaN,Core_RNAlater,185,Diseased_Margin,only margin sample available,092023_BEL_CBC_T1_185_SSID,UML_NARWHAL_R1_B10,NaN,NaN,NaN
2,92023,BEL,CBC,9/25/23,CBC30N,1,NaN,2,PAST,NaN,NaN,Core_RNAlater,171,Bleached_Tissue,CLP 90%,092023_BEL_CBC_T1_171_PAST,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,NaN,NaN
3,92023,BEL,CBC,9/25/23,CBC30N,1,NaN,3,SSID,NaN,NaN,Core_RNAlater,173,Healthy,CLP 80%; DC 20%,092023_BEL_CBC_T1_173_SSID,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,NaN,NaN
4,92023,BEL,CBC,9/25/23,CBC30N,1,NaN,12,PSTR,NaN,NaN,Core_RNAlater,177,Healthy,No CL,092023_BEL_CBC_T1_177_PSTR,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,NaN,NaN


In [37]:
# add leading 0s to monthyear
test['Month_year']=test['Month_year'].astype(str).str.zfill(6)

In [38]:
test.shape
# same dimensions as starting df 

(2529, 20)

In [40]:
# export 
# test.to_csv('/Users/brookesienkiewicz/Documents/sctld/SCTLD_samples/Sample_Data/CBC_samples.csv', index=False)